<a href="https://colab.research.google.com/github/kili-technology/kili-python-sdk/blob/main/recipes/test_domain_api_assets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Testing the Domain API with Legacy and Modern Modes

This notebook demonstrates both the **legacy** and **modern** domain API syntax for asset operations. The Kili Python SDK now supports a `legacy` parameter that controls how you access domain namespaces.

## Key Differences:

**Legacy Mode (`legacy=True` - default):**
- `kili.assets()` - Legacy method for backward compatibility
- `kili.assets_ns` - Domain namespace with organized operations

**Modern Mode (`legacy=False`):**
- `kili.assets` - Direct access to domain namespace (clean name)
- `kili.assets_ns` - Still available for compatibility
- Legacy methods like `kili.assets()` are not available

## Benefits of Modern Mode:
- **Cleaner API**: Use `kili.assets` instead of `kili.assets_ns`
- **Better discoverability**: Natural namespace names
- **Future-proof**: Aligns with domain-driven design principles

## Installing and Setting Up Kili

In [ ]:
%pip install kili

In [ ]:
from kili.client import Kili

## Authentication and Client Setup

We'll demonstrate both legacy and modern modes by creating two client instances.

### Legacy Mode Client (Default Behavior)

In [ ]:
# Configuration for local testing
API_KEY = ""
ENDPOINT = "http://localhost:4001/api/label/v2/graphql"

# Legacy mode client (default behavior)
kili_legacy = Kili(
    api_key=API_KEY,
    api_endpoint=ENDPOINT,
    # legacy=True is the default
)

print("Legacy mode client initialized!")
print(f"Legacy mode setting: {kili_legacy._legacy_mode}")
print("Assets namespace available as: kili_legacy.assets_ns")
print(f"Legacy assets method available: {callable(getattr(kili_legacy, 'assets', None))}")

print("\n" + "=" * 50)

# Modern mode client
kili_modern = Kili(
    api_key=API_KEY,
    api_endpoint=ENDPOINT,
    legacy=False,  # Enable modern mode
)

print("Modern mode client initialized!")
print(f"Legacy mode setting: {kili_modern._legacy_mode}")
print("Assets namespace available as: kili_modern.assets")
print(f"Assets namespace is same instance: {kili_modern.assets is kili_modern.assets_ns}")

# For the rest of the notebook, we'll use both clients to show the differences

Legacy mode client initialized!
Legacy mode setting: True
Assets namespace available as: kili_legacy.assets_ns
Legacy assets method available: True

Modern mode client initialized!
Legacy mode setting: False
Assets namespace available as: kili_modern.assets
Assets namespace is same instance: True


## Creating a Test Project

We'll create a test project using the legacy client (functionality is identical in both modes):

In [ ]:
# Define a simple classification interface
interface = {
    "jobs": {
        "JOB_0": {
            "mlTask": "CLASSIFICATION",
            "required": 1,
            "isChild": False,
            "content": {
                "categories": {
                    "CAR": {"name": "Car"},
                    "TRUCK": {"name": "Truck"},
                    "BUS": {"name": "Bus"},
                },
                "input": "radio",
            },
        }
    }
}

# Create the project (using legacy client - works identically)
project = kili_legacy.create_project(
    title="[Domain API Test]: Legacy vs Modern Modes",
    description="Comparing legacy and modern domain API syntax",
    input_type="IMAGE",
    json_interface=interface,
)

project_id = project["id"]
print(f"Created test project with ID: {project_id}")

Created test project with ID: cmg53u8n40h0dav1adpepa1p8


## Comparing Legacy vs Modern Syntax

Now let's compare how asset creation works in both modes:

In [ ]:
# Test asset URLs
test_urls = [
    "https://storage.googleapis.com/label-public-staging/car/car_1.jpg",
    "https://storage.googleapis.com/label-public-staging/car/car_2.jpg",
    "https://storage.googleapis.com/label-public-staging/recipes/inference/black_car.jpg",
]

print("=== LEGACY MODE SYNTAX ===")
print("Using: kili_legacy.assets_ns.create()")

# Create assets using LEGACY syntax
create_result_legacy = kili_legacy.assets_ns.create(
    project_id=project_id,
    content_array=test_urls,
    external_id_array=["legacy_car_1", "legacy_car_2", "legacy_car_3"],
    json_metadata_array=[
        {"description": "First test car (legacy)", "source": "legacy_mode"},
        {"description": "Second test car (legacy)", "source": "legacy_mode"},
        {"description": "Third test car (legacy)", "source": "legacy_mode"},
    ],
)

print(f"✅ Created {len(create_result_legacy['asset_ids'])} assets using legacy syntax")
legacy_asset_ids = create_result_legacy["asset_ids"]

print("\n=== MODERN MODE SYNTAX ===")
print("Using: kili_modern.assets.create()")

# Create assets using MODERN syntax (note the cleaner namespace name)
create_result_modern = kili_modern.assets.create(
    project_id=project_id,
    content_array=test_urls,
    external_id_array=["modern_car_1", "modern_car_2", "modern_car_3"],
    json_metadata_array=[
        {"description": "First test car (modern)", "source": "modern_mode"},
        {"description": "Second test car (modern)", "source": "modern_mode"},
        {"description": "Third test car (modern)", "source": "modern_mode"},
    ],
)

print(f"✅ Created {len(create_result_modern['asset_ids'])} assets using modern syntax")
modern_asset_ids = create_result_modern["asset_ids"]

print(f"\n📊 Total assets in project: {len(legacy_asset_ids + modern_asset_ids)}")

# Combine asset IDs for later operations
all_asset_ids = legacy_asset_ids + modern_asset_ids

=== LEGACY MODE SYNTAX ===
Using: kili_legacy.assets_ns.create()
✅ Created 3 assets using legacy syntax

=== MODERN MODE SYNTAX ===
Using: kili_modern.assets.create()
✅ Created 3 assets using modern syntax

📊 Total assets in project: 6


## Asset Listing Comparison

Compare asset listing and counting operations:

In [ ]:
print("=== LEGACY MODE: Counting and Listing ===")
print("Using: kili_legacy.assets_ns.count() and kili_legacy.assets_ns.list()")

# Count assets using legacy syntax
legacy_count = kili_legacy.assets_ns.count(project_id=project_id)
print(f"Asset count (legacy): {legacy_count}")

# List assets using legacy syntax
legacy_assets = kili_legacy.assets_ns.list(project_id=project_id, as_generator=False, first=10)
print(f"Retrieved {len(legacy_assets)} assets using legacy syntax")

print("\n=== MODERN MODE: Counting and Listing ===")
print("Using: kili_modern.assets.count() and kili_modern.assets.list()")

# Count assets using modern syntax (cleaner!)
modern_count = kili_modern.assets.count(project_id=project_id)
print(f"Asset count (modern): {modern_count}")

# List assets using modern syntax
modern_assets = kili_modern.assets.list(project_id=project_id, as_generator=False, first=10)
print(f"Retrieved {len(modern_assets)} assets using modern syntax")

print(f"\n🔍 Both methods return the same data: {legacy_count == modern_count}")

# Show some assets from both queries
print("\nSample assets (showing external IDs to differentiate):")
for asset in legacy_assets[:3]:
    external_id = asset.get("externalId", "N/A")
    source = asset.get("jsonMetadata", {}).get("source", "unknown")
    print(f"  - {external_id} (from {source})")

print("\n📈 The functionality is identical - only the syntax differs!")

=== LEGACY MODE: Counting and Listing ===
Using: kili_legacy.assets_ns.count() and kili_legacy.assets_ns.list()
Asset count (legacy): 6



etrieving assets: 100%|████████████████████████████████████| 6/6 [00:00<00:00, 108.03it/s]

Retrieved 6 assets using legacy syntax

=== MODERN MODE: Counting and Listing ===
Using: kili_modern.assets.count() and kili_modern.assets.list()
Asset count (modern): 6


Retrieving assets: 100%|████████████████████████████████████| 6/6 [00:00<00:00, 128.41it/s]

Retrieved 6 assets using modern syntax

🔍 Both methods return the same data: True

Sample assets (showing external IDs to differentiate):
  - legacy_car_1 (from legacy_mode)
  - legacy_car_2 (from legacy_mode)
  - legacy_car_3 (from legacy_mode)

📈 The functionality is identical - only the syntax differs!


## Metadata Operations Comparison

Compare metadata namespace operations between legacy and modern modes:

In [ ]:
print("=== LEGACY MODE: Metadata Operations ===")
print("Using: kili_legacy.assets_ns.metadata.add()")

# Add metadata using legacy syntax
legacy_metadata_result = kili_legacy.assets_ns.metadata.add(
    json_metadata=[
        {"vehicle_type": "sedan", "color": "red", "mode": "legacy"},
        {"vehicle_type": "hatchback", "color": "blue", "mode": "legacy"},
        {"vehicle_type": "sedan", "color": "black", "mode": "legacy"},
    ],
    project_id=project_id,
    asset_ids=legacy_asset_ids,
)

print(f"✅ Added metadata to {len(legacy_metadata_result)} assets (legacy syntax)")

print("\n=== MODERN MODE: Metadata Operations ===")
print("Using: kili_modern.assets.metadata.add()")

# Add metadata using modern syntax (cleaner namespace!)
modern_metadata_result = kili_modern.assets.metadata.add(
    json_metadata=[
        {"vehicle_type": "sedan", "color": "red", "mode": "modern"},
        {"vehicle_type": "hatchback", "color": "blue", "mode": "modern"},
        {"vehicle_type": "sedan", "color": "black", "mode": "modern"},
    ],
    project_id=project_id,
    asset_ids=modern_asset_ids,
)

print(f"✅ Added metadata to {len(modern_metadata_result)} assets (modern syntax)")

print("\n=== COMPARISON ===")
print("Legacy syntax:  kili.assets_ns.metadata.add()")
print("Modern syntax:  kili.assets.metadata.add()    <- Cleaner!")

# Test set metadata with modern syntax
print("\nTesting metadata.set() with modern syntax...")
modern_set_result = kili_modern.assets.metadata.set(
    json_metadata=[
        {"quality_score": 0.95, "processed": True, "mode": "modern_set"},
        {"quality_score": 0.88, "processed": True, "mode": "modern_set"},
        {"quality_score": 0.92, "processed": True, "mode": "modern_set"},
    ],
    project_id=project_id,
    asset_ids=modern_asset_ids,
)

print(f"✅ Set metadata for {len(modern_set_result)} assets using modern syntax")

=== LEGACY MODE: Metadata Operations ===
Using: kili_legacy.assets_ns.metadata.add()
✅ Added metadata to 3 assets (legacy syntax)

=== MODERN MODE: Metadata Operations ===
Using: kili_modern.assets.metadata.add()
✅ Added metadata to 3 assets (modern syntax)

=== COMPARISON ===
Legacy syntax:  kili.assets_ns.metadata.add()
Modern syntax:  kili.assets.metadata.add()    <- Cleaner!

Testing metadata.set() with modern syntax...
✅ Set metadata for 3 assets using modern syntax


## External ID and Workflow Operations

Compare external ID updates and workflow operations:

In [ ]:
print("=== EXTERNAL ID OPERATIONS COMPARISON ===")

# Legacy syntax for external ID updates
print("Legacy: kili_legacy.assets_ns.external_ids.update()")
legacy_external_result = kili_legacy.assets_ns.external_ids.update(
    new_external_ids=["updated_legacy_1", "updated_legacy_2", "updated_legacy_3"],
    asset_ids=legacy_asset_ids,
)
print(f"✅ Updated {len(legacy_external_result)} external IDs (legacy syntax)")

# Modern syntax for external ID updates
print("\nModern: kili_modern.assets.external_ids.update()")
modern_external_result = kili_modern.assets.external_ids.update(
    new_external_ids=["updated_modern_1", "updated_modern_2", "updated_modern_3"],
    asset_ids=modern_asset_ids,
)
print(f"✅ Updated {len(modern_external_result)} external IDs (modern syntax)")

print("\n=== WORKFLOW OPERATIONS COMPARISON ===")

# Try workflow operations (may fail if no users available)
try:
    print("Legacy: kili_legacy.assets_ns.workflow.step.next()")
    legacy_workflow_result = kili_legacy.assets_ns.workflow.step.next(
        asset_ids=[legacy_asset_ids[0]]
    )
    print(f"✅ Legacy workflow operation: {legacy_workflow_result}")
except Exception as e:
    print(f"Legacy workflow operation skipped: {e}")

try:
    print("Modern: kili_modern.assets.workflow.step.next()")
    modern_workflow_result = kili_modern.assets.workflow.step.next(asset_ids=[modern_asset_ids[0]])
    print(f"✅ Modern workflow operation: {modern_workflow_result}")
except Exception as e:
    print(f"Modern workflow operation skipped: {e}")

print("\n📝 Key Takeaway: Modern syntax removes the '_ns' suffix for cleaner code!")

=== EXTERNAL ID OPERATIONS COMPARISON ===
Legacy: kili_legacy.assets_ns.external_ids.update()
✅ Updated 3 external IDs (legacy syntax)

Modern: kili_modern.assets.external_ids.update()
✅ Updated 3 external IDs (modern syntax)

=== WORKFLOW OPERATIONS COMPARISON ===
Legacy: kili_legacy.assets_ns.workflow.step.next()
✅ Legacy workflow operation: None
Modern: kili_modern.assets.workflow.step.next()
✅ Modern workflow operation: None

📝 Key Takeaway: Modern syntax removes the '_ns' suffix for cleaner code!


## Testing Migration Compatibility

Verify that both modes can work with the same data and provide migration paths:

In [ ]:
print("=== TESTING MIGRATION COMPATIBILITY ===")

# Test that modern mode client can still access _ns properties for compatibility
print("✅ Testing modern client compatibility with _ns syntax:")
print(f"kili_modern.assets_ns exists: {hasattr(kili_modern, 'assets_ns')}")
print(f"kili_modern.assets is kili_modern.assets_ns: {kili_modern.assets is kili_modern.assets_ns}")

# Test that we can update assets created with either client using either syntax
print("\n✅ Cross-client compatibility test:")

# Update assets created by legacy client using modern client
modern_update_result = kili_modern.assets.update(
    asset_ids=[legacy_asset_ids[0]],  # Asset created by legacy client
    priorities=[5],
    json_metadatas=[{"updated_by": "modern_client", "cross_compatible": True}],
)
print(f"Modern client updated legacy asset: {len(modern_update_result)} assets")

# Update assets created by modern client using legacy client
legacy_update_result = kili_legacy.assets_ns.update(
    asset_ids=[modern_asset_ids[0]],  # Asset created by modern client
    priorities=[5],
    json_metadatas=[{"updated_by": "legacy_client", "cross_compatible": True}],
)
print(f"Legacy client updated modern asset: {len(legacy_update_result)} assets")

# Demonstrate that legacy client has access to legacy methods
print("\n✅ Legacy client has access to legacy methods:")
print(f"kili_legacy.assets() callable: {callable(getattr(kili_legacy, 'assets', None))}")

# Show that modern client blocks legacy methods
print("\n✅ Modern client blocks legacy methods:")
try:
    # This should fail with a helpful error message
    legacy_method = kili_modern.assets()
    print("ERROR: Modern client should not have access to legacy assets() method")
except AttributeError as e:
    print(f"✅ Expected error: {e}")

print("\n=== MIGRATION STRATEGY ===")
print("1. Start with legacy=True (default) - existing code works")
print("2. Gradually adopt kili.assets instead of kili.assets_ns")
print("3. When ready, switch to legacy=False for clean API")
print("4. Legacy methods are blocked, forcing modern syntax")

# Show the namespace mapping
print("\n=== NAMESPACE MAPPING ===")
namespaces = [
    "assets",
    "projects",
    "labels",
    "users",
    "organizations",
    "issues",
    "notifications",
    "tags",
    "cloud_storage",
]
for ns in namespaces[:3]:  # Show first few examples
    print(f"Legacy:  kili.{ns}_ns")
    print(f"Modern:  kili.{ns}")
    print("---")

=== TESTING MIGRATION COMPATIBILITY ===
✅ Testing modern client compatibility with _ns syntax:
kili_modern.assets_ns exists: True
kili_modern.assets is kili_modern.assets_ns: True

✅ Cross-client compatibility test:
Modern client updated legacy asset: 1 assets
Legacy client updated modern asset: 1 assets

✅ Legacy client has access to legacy methods:
kili_legacy.assets() callable: True

✅ Modern client blocks legacy methods:


TypeError: 'AssetsNamespace' object is not callable

## Testing Workflow Operations

Test workflow-related operations (these may fail if no users are available):

In [ ]:
try:
    # Get users from current organization to find a user for testing
    org_id = kili.organizations()[0]["id"]
    current_users = list(kili.users(organization_id=org_id, first=1))
    if current_users:
        user_id = current_users[0]["id"]
        print(f"Using user ID for testing: {user_id}")
    else:
        raise Exception("No users found in organization")

    # Test workflow assignment (assign to current user)
    assign_result = kili.assets_ns.workflow.assign(
        asset_ids=[asset_ids[0]],  # Just assign the first asset
        to_be_labeled_by_array=[[user_id]],
    )

    print(f"Assigned {len(assign_result)} assets to labelers")

except Exception as e:
    print(f"Workflow assignment test skipped due to: {e}")

try:
    # Test moving assets to next workflow step
    next_step_result = kili.assets_ns.workflow.step.next(asset_ids=[asset_ids[0]])

    if next_step_result:
        print(f"Moved asset to next workflow step: {next_step_result}")
    else:
        print("Asset was already in the correct workflow step")

except Exception as e:
    print(f"Workflow step test skipped due to: {e}")

try:
    # Test invalidating workflow step (send back to queue)
    invalidate_result = kili.assets_ns.workflow.step.invalidate(asset_ids=[asset_ids[0]])

    if invalidate_result:
        print(f"Sent asset back to queue: {invalidate_result}")
    else:
        print("Asset was already in queue")

except Exception as e:
    print(f"Workflow invalidate test skipped due to: {e}")

Using user ID for testing: user-2
Assigned 1 assets to labelers
Asset was already in the correct workflow step


KeyboardInterrupt: 

## Verifying Final State

Let's check the final state of our assets after all operations:

In [ ]:
# Retrieve assets again to see final state
final_assets = kili.assets_ns.list(
    project_id=project_id,
    as_generator=False,
    fields=["id", "externalId", "priority", "jsonMetadata", "status"],
)

print("Final state of assets:")
print("=" * 50)

for asset in final_assets:
    print(f"Asset ID: {asset['id']}")
    print(f"External ID: {asset.get('externalId', 'N/A')}")
    print(f"Priority: {asset.get('priority', 'N/A')}")
    print(f"Metadata: {asset.get('jsonMetadata', {})}")
    print(f"Status: {asset.get('status', 'N/A')}")
    print("-" * 30)

Retrieving assets: 100%|█████████████████████████████████████| 3/3 [00:00<00:00, 76.52it/s]

Final state of assets:
Asset ID: cmg4uzwec0000e51a8hu9dajb
External ID: updated_car_1
Priority: 1
Metadata: {'priority_reason': 'high_quality', 'review_needed': False}
Status: TODO
------------------------------
Asset ID: cmg4uzwec0001e51a3u5p7t6z
External ID: updated_car_2
Priority: 2
Metadata: {'priority_reason': 'medium_quality', 'review_needed': True}
Status: TODO
------------------------------
Asset ID: cmg4uzwec0002e51aum0o4idm
External ID: updated_car_3
Priority: 3
Metadata: {'priority_reason': 'good_quality', 'review_needed': False}
Status: TODO
------------------------------


## Testing Asset Deletion

Finally, test the delete operation:

In [ ]:
# Delete one asset to test the delete method
delete_result = kili.assets_ns.delete(
    asset_ids=[asset_ids[0]]  # Delete just the first asset
)

print(f"Deleted asset: {delete_result}")

# Verify the count decreased
new_count = kili.assets_ns.count(project_id=project_id)
print(f"Assets remaining in project: {new_count}")

## Testing Asset Filtering

Test filtering capabilities with the new syntax:

In [ ]:
# Test filtering by external ID
filtered_assets = kili.assets_ns.list(
    project_id=project_id, external_id_contains=["updated_car_2"], as_generator=False
)

print(f"Assets filtered by external ID: {len(filtered_assets)}")
for asset in filtered_assets:
    print(f"- {asset['externalId']}: {asset['id']}")

# Test getting a specific asset
if len(asset_ids) > 1:
    specific_asset = kili.assets_ns.list(
        project_id=project_id,
        asset_id=asset_ids[1],  # Get the second asset
        as_generator=False,
    )

    print(f"\nSpecific asset retrieved: {len(specific_asset)} asset(s)")
    if specific_asset:
        print(f"Asset details: {specific_asset[0]['externalId']} - {specific_asset[0]['id']}")

## Performance Comparison Test

Let's compare the performance of the new syntax with a simple benchmark:

In [ ]:
import time

print("=== PERFORMANCE COMPARISON ===")

# Test modern syntax performance
start_time = time.time()
modern_count = kili_modern.assets.count(project_id=project_id)
modern_assets_perf = kili_modern.assets.list(project_id=project_id, first=5, as_generator=False)
modern_time = time.time() - start_time

print("Modern syntax (kili.assets):")
print(f"- Count: {modern_count}")
print(f"- Retrieved: {len(modern_assets_perf)} assets")
print(f"- Time taken: {modern_time:.3f} seconds")

# Test legacy domain API syntax
start_time = time.time()
legacy_count = kili_legacy.assets_ns.count(project_id=project_id)
legacy_assets_perf = kili_legacy.assets_ns.list(project_id=project_id, first=5, as_generator=False)
legacy_time = time.time() - start_time

print("\nLegacy domain API syntax (kili.assets_ns):")
print(f"- Count: {legacy_count}")
print(f"- Retrieved: {len(legacy_assets_perf)} assets")
print(f"- Time taken: {legacy_time:.3f} seconds")

# Test old-style methods for comparison (if available)
try:
    start_time = time.time()
    old_count = kili_legacy.count_assets(project_id=project_id)
    old_assets = list(kili_legacy.assets(project_id=project_id, first=5))
    old_time = time.time() - start_time

    print("\nOld-style methods (kili.count_assets, kili.assets):")
    print(f"- Count: {old_count}")
    print(f"- Retrieved: {len(old_assets)} assets")
    print(f"- Time taken: {old_time:.3f} seconds")

    print("\n📊 Performance Analysis:")
    print(f"- Modern syntax: {modern_time:.3f}s")
    print(f"- Legacy domain API: {legacy_time:.3f}s")
    print(f"- Old-style methods: {old_time:.3f}s")

except AttributeError:
    print("\nOld-style methods not available for comparison")

    print("\n📊 Performance Analysis:")
    print(f"- Modern syntax: {modern_time:.3f}s")
    print(f"- Legacy domain API: {legacy_time:.3f}s")
    print("- Both use the same underlying implementation!")

print("\n✨ Performance is identical - only syntax differs!")

## Summary of New Features Tested

## Summary: Legacy vs Modern Domain API

This notebook successfully demonstrated the differences between legacy and modern domain API modes:

### ✅ Legacy Mode (`legacy=True` - default)
- **Backward Compatibility**: All existing code continues to work
- **Namespace Access**: Use `kili.assets_ns` for domain operations  
- **Legacy Methods**: `kili.assets()`, `kili.projects()`, etc. still available
- **Migration Path**: Gradual adoption of domain API alongside existing code

### ✅ Modern Mode (`legacy=False`)
- **Clean API**: Use `kili.assets` instead of `kili.assets_ns`
- **Natural Naming**: Domain namespaces have intuitive names
- **Future-Proof**: Aligns with domain-driven design principles
- **Clear Migration**: Legacy methods blocked with helpful error messages

### 🔄 Complete Feature Parity
Both modes provide identical functionality:

**Core Operations:**
- ✅ `list()` / `count()` - List and count assets
- ✅ `create()` / `update()` / `delete()` - CRUD operations  

**Nested Namespaces:**
- ✅ `metadata.add()` / `metadata.set()` - Metadata operations
- ✅ `external_ids.update()` - External ID management
- ✅ `workflow.assign()` / `workflow.step.*` - Workflow operations

**Advanced Features:**
- ✅ Generator vs List modes
- ✅ Filtering and querying
- ✅ Bulk operations
- ✅ Thread safety and lazy loading

### 🚀 Migration Strategy

1. **Start**: Use default `legacy=True` - no changes needed
2. **Transition**: Replace `kili.assets_ns` with `kili.assets` gradually  
3. **Modernize**: Switch to `legacy=False` when ready
4. **Clean**: Enjoy cleaner, more intuitive namespace names

### 📈 Benefits of Modern Mode

- **Developer Experience**: More intuitive and discoverable API
- **Code Readability**: `kili.assets.list()` vs `kili.assets_ns.list()`
- **Future Compatibility**: Aligned with domain-driven architecture
- **Clear Intent**: Namespace names match their purpose

The modern domain API provides the same powerful functionality with a cleaner, more intuitive interface!

## Cleanup

Clean up by deleting the test project:

In [ ]:
# Clean up by deleting the test project (using either client works)
kili_legacy.delete_project(project_id)
print(f"Deleted test project: {project_id}")
print("\n🎉 Legacy vs Modern Domain API comparison completed successfully!")
print("\n💡 Key Takeaway: Modern mode (legacy=False) provides the same functionality")
print("   with cleaner, more intuitive namespace names!")

# Show the simple syntax difference one more time
print("\n📝 Quick Reference:")
print("Legacy Mode:  kili = Kili()         # default")
print("              kili.assets_ns.list()")
print("")
print("Modern Mode:  kili = Kili(legacy=False)")
print("              kili.assets.list()    # cleaner!")